In [1]:
import json
import numpy as np
import cvxpy as cp
with open("Recon3D.json", "r") as file_handle:
    dictionary = json.load(file_handle);
    data = dictionary["reactions"];
    temp_name = [(data[i])["name"] for i in range(len(data))];
    b = [i for i in range(len(data)) if temp_name[i] == "Generic Human Biomass Reaction"];
    b = b[0];
    order = [i for j in (range(b), range(b+1, len(data)), range(b,b+1)) for i in j];
    name = [(data[i])["name"] for i in order];
    lower_bound = [(data[i])["lower_bound"] for i in order];
    upper_bound = [(data[i])["upper_bound"] for i in order];
    subsystem = [(data[i])["subsystem"] for i in order];
    metabolites = [(data[i])["metabolites"] for i in order];
    id = [((dictionary["metabolites"])[i])["id"] for i in range(len(dictionary["metabolites"]))];
    S = np.zeros((len(id), len(metabolites)));
    for i in range(len(metabolites)):
        for j in range(len(id)):
            if id[j] in metabolites[i].keys():
                S[j, i] = metabolites[i][id[j]];
knockout_names = [
        'Transport, nuclear',
        'Fatty acid oxidation',
        ]

In [30]:
def solve_with_removed_index(removed=None):
    solvers = {'ECOS': cp.ECOS, 'QSOP': cp.OSQP, 'SCS': cp.SCS}
    for solver_name, solver in solvers.items():
        u = np.array(upper_bound.copy())
        l = np.array(lower_bound.copy())
        if removed is not None:
            removed = np.array(removed)
            u[removed] = 0
            l[removed] = 0
        m, n = S.shape
        v = cp.Variable(n)
        constraints = [
                S @ v == 0,
                v <= u,
                v >= l,
                ]
        obj = cp.Maximize(v[-1])
        problem = cp.Problem(obj, constraints)
        ans = problem.solve(solver=solver)
        if problem.status == 'optimal':
            return problem, v.value, solver_name

def get_knockout_index(knockout):
    return [i for i, sub_name in enumerate(subsystem) if sub_name in knockout]

def get_subystem_indexes(name):
    return get_knockout_index({name})

def part_a():
    #should run read_data before this
    p_a, v_a, solver_a = solve_with_removed_index()
    return p_a, v_a, solver_a

def evaluated_diff(v_w, v_tilde):
    return (v_w[-1] - v_tilde[-1]) / v_w[-1]

def part_b():
    p_transport, v_transport, solver_transport = solve_with_removed_index(get_subystem_indexes(knockout_names[0]))
    p_fatty, v_fatty, solver_fatty = solve_with_removed_index(get_subystem_indexes(knockout_names[1]))
    return p_transport, v_transport, solver_transport, p_fatty, v_fatty, solver_fatty

def part_c(v_a):
    possible_indexes = get_subystem_indexes(knockout_names[0])
    print('length of list: ', len(possible_indexes))
    li = []
    for i, index in enumerate(possible_indexes):
        print(f'starting {i}th index {index}, name={name[index]}')
        p_index, v_index, solver_index = solve_with_removed_index([index])
        evaluation = evaluated_diff(v_a, v_index)
        made_it = evaluation > 0.2
        print(f'status: {p_index.status}, solver: {solver_index}, evaluation: {evaluation}, made it: {made_it}')
        if made_it:
            li.append(name[index])
    return li

def main():
    p_a, v_a, solver_a = part_a()
    p_transport, v_transport, solver_transport, p_fatty, v_fatty, solver_fatty = part_b()
    print(p_a.status, p_transport.status, p_fatty.status)
    print(p_a.value, p_transport.value, p_fatty.value)
    print(evaluated_diff(v_a, v_transport), evaluated_diff(v_a, v_fatty))

In [31]:
li_c = part_c(v_a)

length of list:  71
starting 0th index 50, name=35cGMP nuclear transport
status: optimal, solver: ECOS, evaluation: -8.743789840439052e-13, made it: False
starting 1th index 161, name=Acetyl CoA transport  nuclear
status: optimal, solver: ECOS, evaluation: -3.44982802472276e-13, made it: False
starting 2th index 184, name=Acetylcholin transport, nuclear trhough pores
status: optimal, solver: ECOS, evaluation: -3.4453006913569824e-13, made it: False
starting 3th index 199, name=N-acetylneuraminate nuclear import
status: optimal, solver: ECOS, evaluation: -8.266910725910447e-13, made it: False
starting 4th index 386, name=ATP diffusion in nucleus
status: optimal, solver: ECOS, evaluation: -1.720084856771217e-12, made it: False
starting 5th index 439, name=Biocytin transport, nuclear
status: optimal, solver: ECOS, evaluation: -1.2970810092953686e-12, made it: False
starting 6th index 452, name=Biotin transport, nuclear
status: optimal, solver: ECOS, evaluation: 8.88262806365624e-13, made 

status: optimal, solver: ECOS, evaluation: -5.252461259863328e-12, made it: False
starting 57th index 8202, name=Sphinganine Intracellular Transport
status: optimal, solver: ECOS, evaluation: -5.716512929855562e-13, made it: False
starting 58th index 8204, name=Sphingosine Intracellular Transport
status: optimal, solver: ECOS, evaluation: -6.013807820874978e-13, made it: False
starting 59th index 8211, name=Transport of Gd1A 
status: optimal, solver: ECOS, evaluation: -7.436899608851165e-13, made it: False
starting 60th index 8226, name=Galactocerebroside Intracellular Transport
status: optimal, solver: ECOS, evaluation: 0.003470691713837586, made it: False
starting 61th index 8227, name=Ceramide Transport Protei
status: optimal, solver: ECOS, evaluation: 0.003470691713837435, made it: False
starting 62th index 8232, name=Transport of Gd1A 
status: optimal, solver: ECOS, evaluation: -9.345925178087513e-13, made it: False
starting 63th index 8552, name=HMR 0984
status: optimal, solver: 

In [32]:
li_c

['DATP diffusion in nucleus', 'DGTP diffusion in nucleus']

In [12]:
main()

optimal optimal optimal
753.3364247990297 0.0007596634022799669 753.3364248096925
0.9999989916013918 -1.4154255034768028e-11
